# ООП-2

### Глава 1: датаклассы и енамы

В датаклассе не нужен инит, достаточно просто объявит нужны в нем поля и дальше так и использовать

In [11]:
from dataclasses import dataclass

@dataclass
class User:
    email: str
    gender: int  # 1 for male, 0 for female
    first_name: str
    middle_name: str
    last_name: str

    @property
    def full_name(self):
        return f"{self.first_name} {self.middle_name} {self.last_name}"

In [12]:
user = User("astreltsov@hse.ru", 1, "Артем", "Дмитриевич", "Стрельцов")
# user = User(email="astreltsov@hse.ru", gender=1, first_name="Артем", middle_name="Дмитриевич", last_name="Стрельцов")
print(user.email)
print(user.full_name)

astreltsov@hse.ru
Артем Дмитриевич Стрельцов


Из грустного: gender сейчас хранится числом и получается какая-то магия, постоянно надо помнить, какому полу соответствует какое число

In [27]:
from dataclasses import dataclass
from enum import Enum, auto


class Gender(Enum):  # тут также можно использовать IntEnum
    female = auto()  # auto автоматически пронумерует элементы енама
    male = auto()  # но можно задать какое-то и свое значение, например
                   # male = "male"
                   # female = "female"

In [31]:
print(Gender.male)
print(Gender.male.name)
print(Gender.male.value)

male
male
2


In [41]:
Gender._member_map_

{'female': female, 'male': male}

In [38]:
Gender._value2member_map_

{1: female, 2: male}

In [35]:
from dataclasses import dataclass
from enum import Enum, auto


class Gender(Enum):  # тут также можно использовать IntEnum
    female = auto()
    male = auto()

    def __str__(self):
        return self.name  # name, чтобы получить названия самого элемента енама, value -- чтобы получить значение

    __repr__ = __str__

In [42]:
from dataclasses import dataclass

@dataclass
class User:
    email: str
    gender: Gender
    first_name: str
    middle_name: str
    last_name: str

    @property
    def full_name(self):
        return f"{self.first_name} {self.middle_name} {self.last_name}"

    @property
    def is_male(self):
        return self.gender == Gender.male

In [43]:
user = User(email="astreltsov@hse.ru", gender=Gender.male, first_name="Артем", middle_name="Дмитриевич", last_name="Стрельцов")
print(user.email)
print(user.full_name)
print(user.gender)
print(user.is_male)

astreltsov@hse.ru
Артем Дмитриевич Стрельцов
male
True


### Глава 2: классметоды и статикметоды

Classmethod используется когда нужно вызвать функцию класса, не создавая объект. Чаще всего это нужно для создания экземпляра класса на основе каких-то данных которые еще как-то нужно обработать

In [60]:
from dataclasses import dataclass
from datetime import date

@dataclass
class Person:
    age: int
    name: str

    @classmethod
    def from_birthyear(cls, name: str, birth_year: int) -> "Person":  # в cls лежит фактически сам класс
        return cls(
            age=date.today().year - birth_year,
            name=name,
        )

    def __str__(self):
        return f"{self.name}, возраст {self.age}"

In [46]:
tema = Person.from_birthyear("Тёма", 2000)

print(tema)

Тёма, возраст 22


Staticmethod же применяется обычно для инкапуляции какой-то логики внутри класса, которой вроде и не нужен self, а вроде и не хочется это просто отдельными функциями плодить

In [61]:
from dataclasses import dataclass
from datetime import date

@dataclass
class Person:
    age: int
    name: str

    @staticmethod
    def calculate_age(birth_year: int) -> int:  # заметьте, тут не нужен self/cls
        return date.today().year - birth_year

    @classmethod
    def from_birthyear(cls, name: str, birth_year: int) -> "Person":
        return cls(
            age=cls.calculate_age(birth_year),  # а вот тут уже нужен
            name=name,
        )

    def __str__(self):
        return f"{self.name}, возраст {self.age}"

In [62]:
tema = Person.from_birthyear("Тёма", 2000)

print(tema)

Тёма, возраст 22


### Глава 3: (не)безопасность

В питоне есть так называемые private переменные класса, но работают они с переменным успехом.

In [88]:
from dataclasses import dataclass


@dataclass
class Client:
    def __init__(self, id, phone_number):
        self.id = id
        self.__phone_number = phone_number

In [90]:
client = Client(123, "88005553535")
client.__phone_number  # выдаст ошибку, так как переменная приватная

AttributeError: 'Client' object has no attribute '__phone_number'

НО!!!

In [91]:
client.__dict__  # ой, а как так вышло?)

{'id': 123, '_Client__phone_number': '88005553535'}

In [92]:
client._Client__phone_number

'88005553535'

### Глава 4: namedtuple

Это некоторый гибрид кортежа и класса. Работает как обычный кортеж, только помимо этого у него именованные поля.

In [93]:
from collections import namedtuple

Point = namedtuple("Point", ["x", "y"])

w = Point(5, 6)

print(w)

Point(x=5, y=6)


В остальном это обычный tuple:

In [98]:
for coord in w:
    print(coord)

5
6


In [99]:
w.count(5)

1

In [100]:
len(w)

2